In [135]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)


import matplotlib.pyplot as plt
import seaborn as sns

# from base.JuUnits import parallal_task,task_chunk_split
# from base.JuUnits import excute_for_multidates

# from sklearn.decomposition import FactorAnalysis
# from factor_analyzer import FactorAnalyzer
# from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity,calculate_kmo

# import inspect
import warnings
from sklearn.model_selection import train_test_split

from catboost import CatBoostRanker, Pool, MetricVisualizer


In [16]:
file_names = ['base10','alpha42','alpha54','alpha60','alpha77','191_alpha004', '191_alpha010', '191_alpha021', '191_alpha070', '191_alpha095', '191_alpha115', 
 '191_alpha124', '191_alpha126', '191_alpha129', '191_alpha132', '191_alpha143', '191_alpha144', 
  '191_alpha146', '191_alpha150', '191_alpha153', '191_alpha160', '191_alpha161', '191_alpha173', '191_alpha187']

temp = []
for file in file_names[0:7]:
    print(file)
    temp.append(load_cache(file+'_train',cache_type=CACHE_TYPE.FACTOR)
)

data_all = pd.concat(temp,axis=1).sort_index()#.loc[(slice('2017-01-11','2020-12-24'),codes),:]



In [85]:
data = data_all.loc[(slice('2018-01-11','2020-12-24'),slice(None))]

In [91]:
# codes = data.index.get_level_values(1).unique()
# dates = data.index.get_level_values(0).unique()
# data_replenish = load_cache_adv('all_train_qfq',
#                                 dates.min().strftime('%Y-%m-%d'),
#                                 dates.max().strftime('%Y-%m-%d'),
#                                 cache_type=CACHE_TYPE.STOCK)

# data_replenish = data_replenish.loc[(slice(None),data_replenish.index.get_level_values(1).unique().intersection(codes)),]

# ret = smpl.get_forward_return(data_replenish,'close')
# ret.name = 'target'
# ret[np.abs(ret)>0.15]=np.nan

# industy = data_replenish['industry']
# industy.name = 'industy'
# cap_log = np.log(data_replenish['market_value'])
# cap_log.name = 'cap_log'

# data = pd.concat([data,industy,cap_log,ret],axis=1).sort_index().dropna()
# train_data = data[data.columns[:-1]]
# train_target = data[[data.columns[-1]]]

In [130]:
X_train,X_test, y_train, y_test =train_test_split(train_data,train_target,test_size=.2, shuffle=False)
X_train_group = y_train.groupby(level=0).count()
y_test_group = y_test.groupby(level=0).count()

In [122]:
# y_test['day']= y_test.index.get_level_values(0)


,target
date,
2020-06-29,2474
2020-06-30,2591
2020-07-01,2594
2020-07-02,2594
2020-07-03,2594
...,...
2020-12-17,2631
2020-12-18,2630
2020-12-21,2632


In [117]:
y_test.groupby(level=0).count() 

,target,day
date,,
2020-06-29,2474,2474
2020-06-30,2591,2591
2020-07-01,2594,2594
2020-07-02,2594,2594
2020-07-03,2594,2594
...,...,...
2020-12-17,2631,2631
2020-12-18,2630,2630
2020-12-21,2632,2632
